In [ ]:
##import 
import FlowCytometryTools
from FlowCytometryTools import FCMeasurement
from FlowCytometryTools import ThresholdGate, PolyGate
from pylab import *
from itertools import chain
import glob
import matplotlib.pylab as pylab
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
params = {'legend.fontsize': 'x-large',
#           'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)
import pandas as pd 
from FlowCytometryTools import PolyGate
%matplotlib inline
import seaborn as sns
import math 

In [ ]:
#import flowcytometry tools
from FlowCytometryTools import FCMeasurement

In [ ]:
#import os to change to directory with FC data
import os
datadir = '<name of directory'
os.chdir (datadir)

In [ ]:
#get all FC files in directory
files=glob.glob("*.fcs")
datacounts = pd.DataFrame(columns = ['Name', 'Counts', 'Ref']) 

In [ ]:
# set a sample to find the limits for gating
for i in range(0,1):
    sample = FCMeasurement(ID='Test Sample', datafile=files[i])
print(sample.channel_names)

In [ ]:
# look at FSC-A and SSC-A for gating cells
sample.plot(['FSC-A', 'SSC-A'], bins=100, alpha=0.9, cmap=cm.hot);
plot.ylim(-2, 2)([0,20000])


In [ ]:
# select the gate, interactive interface, hit close when done
sample.view_interactively(backend='wx')

In [ ]:
# copy the gate coordinates and create a gate call 'cells', change region to 'in'
cells = PolyGate([(5.060e+03, 1.503e+04), (1.951e+04, 1.691e+04), (2.866e+04, 1.503e+04), (3.107e+04, 3.769e+03), (2.192e+04, 9.538e+02), (1.084e+04, 9.538e+02), (4.097e+03, 9.538e+02), (5.542e+03, 1.221e+04)], ('FSC-A', 'SSC-A'), region='in', name='gate1')


In [ ]:
# gate the sample and visualize it
gated_sample = sample.gate(cells)
gated_sample.plot(['FSC-A', 'SSC-A']);

In [ ]:
gated_sample.plot(['FITC-A'], bins=300, alpha=0.5);
plt.xticks(rotation=45)
plt.title(sample.datafile)

In [ ]:
#generate histogram for each file 
for i in range(0,len(files)):
    plt.figure(i, figsize=(5,5))
    sample = FCMeasurement(ID='Sample', datafile=files[i])
    gated_sample = sample.gate(cells)
    plt.hist(gated_sample.data[['FITC-A']][:].values, bins = 300, alpha = 0.7)
    plt.title(sample.datafile)
    plt.tick_params('x', labelrotation=45)
    plt.tight_layout()
    plt.savefig(sample.datafile + '.pdf')

In [ ]:
#generate smoothed histogram 
for i in range(0,len(files)):
    plt.figure(i, figsize=(5,5))
    sample = FCMeasurement(ID='Sample', datafile=files[i])
    gated_sample = sample.gate(cells)
    sns.distplot(gated_sample.data[['FITC-A']][:].values, hist = False, kde = True,
                 kde_kws = {'shade': True, 'linewidth': 3}, label = 'arrest')
    plt.title(sample.datafile)
    plt.tick_params('x', labelrotation=45)
    plt.tight_layout()

    plt.savefig(sample.datafile + '_smoothed.pdf')

In [ ]:
#make list of only WT values
filesWT = [i for i in files if not ('57' in i)]
#calculate average alpha factor median score (depends on format of FSC file 3.0 vs 2.0)
WTbackground = 51449
#set up loop conditions. counter will be used as x-axis in scatter plot, dfmedian will be y-value
counter = 0 
WTmedian = []
counterlist = []
for i in range(0,len(filesWT)):
    sample = FCMeasurement(ID='Sample', datafile=filesWT[i])
    gated_sample = sample.gate(cells)
    #x = pd.DataFrame({'FITC' : gated_sample.data['FITC-A']})
    x = gated_sample.data['FITC-A'].median()/WTbackground
    if '_a.' in filesWT[i]:
        counter = counter + 1   
    WTmedian.append(x)
    counterlist.append(counter)
    
#make list of only YW values
filesYW = [i for i in files if ('57' in i)]
#calculate average alpha factor median score
YWbackground = 53519
#set up loop conditions. counter will be used as x-axis in scatter plot, dfmedian will be y-value
counter = 0 
YWmedian = []
counterlist = []
for i in range(0,len(filesYW)):
    sample = FCMeasurement(ID='Sample', datafile=filesYW[i])
    gated_sample = sample.gate(cells)
    #x = pd.DataFrame({'FITC' : gated_sample.data['FITC-A']})
    x = gated_sample.data['FITC-A'].median()/YWbackground
    if '_a.' in filesWT[i]:
        counter = counter + 1   
    YWmedian.append(x)
    counterlist.append(counter)
    
#make list of only 50 values
filess50 = [i for i in files50 if ('123' in i)]
filess50 = sort(filess50)
#calculate average alpha factor median score
WT50background = 51605
#set up loop conditions. counter will be used as x-axis in scatter plot, dfmedian will be y-value
counter = 0 
s50median = []
counterlistC = (3,4,5,6,7,8,9,10,11,12,13,14,15,2,1,3,4,5,6,7,8,9,10,11,12,13,14,15,2,1,3,4,5,6,7,8,9,10,11,12,13,14,15,2,1)
for i in range(0,len(filess50)):
    sample = FCMeasurement(ID='Sample', datafile=filess50[i])
    gated_sample = sample.gate(cells)
    #x = pd.DataFrame({'FITC' : gated_sample.data['FITC-A']})
    x = gated_sample.data['FITC-A'].median()/WT50background 
    s50median.append(x)

In [ ]:
#calc median, min, max, stdev for replicates
import statistics

modcounterlistWT = (0,1,2,4,5,6,7,8,9,10,11,12,14,16,18) 
modcounterlist50 = (2,4,5,6,7,8,9,10,11,12,14,16,18,1,0) 

WTmedianAVG = []
WTmedianmin = []
WTmedianmax = []
WTmedianstdev = []
counter = 0
for i in range(0,len(WTmedian),2):
    x = mean(WTmedian[counter:(counter +2)])
    minx = min(WTmedian[counter:(counter +2)])
    maxx = max(WTmedian[counter:(counter +2)])
    stdevx = statistics.stdev(WTmedian[counter:(counter +2)])
    counter = counter + 2
    WTmedianAVG.append(x)
    WTmedianmin.append(minx)
    WTmedianmax.append(maxx)
    WTmedianstdev.append(stdevx)
    
YWmedianAVG = []
YWmedianmin = []
YWmedianmax = []
YWmedianstdev = []
counter = 0
for i in range(0,len(YWmedian),2):
    x = mean(YWmedian[counter:(counter +2)])
    minx = min(YWmedian[counter:(counter +2)])
    maxx = max(YWmedian[counter:(counter +2)])
    stdevx = statistics.stdev(YWmedian[counter:(counter +2)])
    counter = counter + 2
    YWmedianAVG.append(x)
    YWmedianmin.append(minx)
    YWmedianmax.append(maxx)
    YWmedianstdev.append(stdevx)
    
s50medianAVG = []
s50medianmin = []
s50medianmax = []
s50medianstdev = []
counter = 0
for i in range(0,15):
    samples = (((s50median[counter]),(s50median[counter+15]),(s50median[counter+30])))
    x = math.fsum(samples)/3
    minx = min(samples)
    maxx = max(samples)
    stdevx = statistics.stdev(samples)
    counter = counter + 1
    s50medianAVG.append(x) 
    s50medianmin.append(minx)
    s50medianmax.append(maxx)
    s50medianstdev.append(stdevx)
    
order = [14,13,0,1,2,3,4,5,6,7,8,9,10,11,12]
s50medianAVG_sorted = [s50medianAVG[i] for i in order]
s50medianmin_sorted = [s50medianmin[i] for i in order]
s50medianmax_sorted = [s50medianmax[i] for i in order]
s50medianstdev_sorted = [s50medianstdev[i] for i in order]


In [ ]:
#plot line of median signal with std error 
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

fig, ax = plt.subplots(1,1)
plt.plot(modcounterlistWT[1:15], WTmedianAVG[1:15], c='gray', label='WT', marker='o')
plt.errorbar(modcounterlistWT[1:15], WTmedianAVG[1:15], yerr = WTmedianstdev[1:15],fmt='o',ecolor = 'gray', color = 'gray')
plt.plot(modcounterlistWT[1:15], YWmedianAVG[1:15], c='skyblue', label='YW', marker='o')
plt.errorbar(modcounterlistWT[1:15], YWmedianAVG[1:15], yerr = YWmedianstdev[1:15],fmt='o',ecolor = 'skyblue', color = 'skyblue')
plt.plot(modcounterlistWT[1:15], s50medianAVG_sorted[1:15], c='blue', label='50', marker='o')
plt.errorbar(modcounterlistWT[1:16], s50medianAVG_sorted[1:16], yerr = s50medianstdev_sorted[1:16],fmt='o',ecolor = 'blue', color = 'blue')
x_ticks_labels = ['','alpha','10','20','25','30','35','40','45','50','55','60','70','80','90']
ax.set_xticklabels(x_ticks_labels, rotation='vertical')
ax.xaxis.set_ticks(modcounterlistWT)
plt.ylabel("Median SYTOX Green Value")
plt.xlabel("time")
plt.legend(loc='lower right')
plt.savefig('/Volumes/ExFAT/DATA/Flow_Cytometry/20220213_2_WT.YW.50selecttimes/WT.YW.s50_allreplicates_blues.line_stdev.svg')
plt.show()


In [ ]:
#isolate alpha only samples (note depends on order of files)
filesalpha = [i for i in filesexample if ('alpha' in i)]
sample50 = FCMeasurement(ID='Sample', datafile=filesalpha[2])
gated_sample50 = sample50.gate(cells)
sampleWT = FCMeasurement(ID='Sample', datafile=filesalpha[0])
gated_sampleWT = sampleWT.gate(cells)
sampleYW = FCMeasurement(ID='Sample', datafile=filesalpha[1])
gated_sampleYW = sampleYW.gate(cells)

In [ ]:
#isolate 30 minute samples (note depends on order of files)
filesalpha = [i for i in filesexample if ('30' in i)]
sample50 = FCMeasurement(ID='Sample', datafile=filesalpha[1])
gated_sample50 = sample50.gate(cells)
sampleWT = FCMeasurement(ID='Sample', datafile=filesalpha[0])
gated_sampleWT = sampleWT.gate(cells)
sampleYW = FCMeasurement(ID='Sample', datafile=filesalpha[2])
gated_sampleYW = sampleYW.gate(cells)

In [ ]:
#isolate 45 minute samples (note depends on order of files)
filesalpha = [i for i in filesexample if ('45' in i)]
sample50 = FCMeasurement(ID='Sample', datafile=filesalpha[0])
gated_sample50 = sample50.gate(cells)
sampleWT = FCMeasurement(ID='Sample', datafile=filesalpha[1])
gated_sampleWT = sampleWT.gate(cells)
sampleYW = FCMeasurement(ID='Sample', datafile=filesalpha[2])
gated_sampleYW = sampleYW.gate(cells)

In [ ]:
#isolate 60 minute samples (note depends on order of files)
filesalpha = [i for i in filesexample if ('60' in i)]
sample50 = FCMeasurement(ID='Sample', datafile=filesalpha[0])
gated_sample50 = sample50.gate(cells)
sampleWT = FCMeasurement(ID='Sample', datafile=filesalpha[1])
gated_sampleWT = sampleWT.gate(cells)
sampleYW = FCMeasurement(ID='Sample', datafile=filesalpha[2])
gated_sampleYW = sampleYW.gate(cells)

In [ ]:
#plot - make sure to update title based on plot
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

plt.hist(gated_sampleWT.data[['FITC-A']][:].values, bins = 300, alpha = 0.7, label = "WT", color = 'gray')
plt.hist(gated_sampleYW.data[['FITC-A']][:].values, bins = 300, alpha = 0.7, label = "YW", color = 'skyblue')
plt.hist(gated_sample50.data[['FITC-A']][1:50000].values, bins = 300, alpha = 0.7, label = "50", color = 'mediumblue')

plt.title("alpha")
plt.tick_params('x', labelrotation=45)
plt.legend(loc='upper right')
plt.xlim([10000, 150000])
plt.savefig('/Volumes/ExFAT/DATA/Flow_Cytometry/20220213_2_WT.YW.50selecttimes/WT.s50_blues.alpha_histogram.svg')
plt.tight_layout()